# Open3D Showcase

The intention for this notebook is to showcase functionalities of the Open3D library which could be useful for the 3D plant estimation project.

Author: David Helm

In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Reading Point Clouds

In [15]:
pcd = o3d.io.read_point_cloud("../data/processed/init_testing/pot01_opensfm.ply", format='xyzn')

In [26]:
print(type(pcd))
print(np.asarray(pcd.colors))

<class 'open3d.cpu.pybind.geometry.PointCloud'>
[]


### Visualizing a Point Cloud

The visualization settings can be left blank, in this case the viewer chooses an ok position by itself.

In [ ]:
# vis settings:
zoom=0.4812
front=[ -0.026201836443756708, -0.053754127166582288, 0.99821037741526886 ]
lookat=[ 0.94675979736194682, 0.31918421391035429, 0.042039549112648671 ]
up=[-0.0694, -0.9768, 0.2024]
# visualize point cloud
o3d.visualization.draw_geometries([pcd],
                                  zoom=zoom,
                                  front=front,
                                  lookat=lookat,
                                  up=up)

In [5]:
o3d.visualization.draw_geometries([pcd])

## Processing
In the next steps some processing functions are shown

### Voxel Downsampling


In [6]:
# voxel downsampling
downpcd = pcd.voxel_down_sample(voxel_size=0.05)
o3d.visualization.draw_geometries([downpcd])

### DBSCAN

In [7]:
# todo: make this function--> dbscan(pcd)
pcd_dbscan = pcd
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd_dbscan.cluster_dbscan(eps=0.02, min_points=80,print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd_dbscan.colors = o3d.utility.Vector3dVector(colors[:, :3])


[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 87
point cloud has 87 clusters


In [8]:
o3d.visualization.draw_geometries([pcd_dbscan])

### outlier removal
Open3D provides multiple outlier removal techniques:
- Statistical Outlier Removal
- Radius Outlier Removal


In [55]:
# input
print("Load a ply point cloud, print it, and render it")
pcd = o3d.io.read_point_cloud("../data/processed/init_testing/pot01_opensfm.ply")
o3d.visualization.draw_geometries([pcd])

print("Downsample the point cloud with a voxel of 0.02")
voxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.002)
o3d.visualization.draw_geometries([voxel_down_pcd])

Load a ply point cloud, print it, and render it
Downsample the point cloud with a voxel of 0.02


In [60]:
# helper
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (green): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0, 1, 0])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

### Statistical outlier removal

In [61]:
print("Statistical oulier removal")
cl, ind = voxel_down_pcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=2.0)
display_inlier_outlier(voxel_down_pcd, ind)

Statistical oulier removal
Showing outliers (red) and inliers (green): 


In [62]:
inlier_cloud = voxel_down_pcd.select_by_index(ind)

### Radius outlier removal

In [67]:
print("Radius oulier removal")
cl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=16, radius=0.05)
display_inlier_outlier(voxel_down_pcd, ind)
radius_inlier_cloud = voxel_down_pcd.select_by_index(ind)

Radius oulier removal
Showing outliers (red) and inliers (green): 


## Surface Reconstruction
http://www.open3d.org/docs/release/tutorial/geometry/surface_reconstruction.html

## Ball Pivoting

In [ ]:
radii = [0.01, 0.02, 0.04]
reconstruction_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    radius_inlier_cloud, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([radius_inlier_cloud, reconstruction_mesh])

In [ ]:
o3d.visualization.draw_geometries([reconstruction_mesh])